In [1]:
import numpy as np
import tensorflow as tf
from functools import wraps
from tensorflow import keras
from sklearn.datasets import load_iris
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

#Importiere MINST Daten
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets('data/', one_hot=True)

#mint=tf.keras.datasets.mnist
#(x_,y_),(x_1,y_1)=mnist.load_data()

from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

## Funktionen generieren und Accuracy auswerten

In [2]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [9]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Verarbeiten der Daten
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reservieren 10.000 Samples zur Validierung
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [10]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [11]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

Fit model on training data
Epoch 1/2
782/782 [==============================] - 4s 6ms/step - loss: 0.0722 - sparse_categorical_accuracy: 0.9785 - val_loss: 0.1029 - val_sparse_categorical_accuracy: 0.9697
Epoch 2/2
782/782 [==============================] - 4s 5ms/step - loss: 0.0599 - sparse_categorical_accuracy: 0.9818 - val_loss: 0.0947 - val_sparse_categorical_accuracy: 0.9725


In [13]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='Testdatenfile.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args):
        #logging.info('Ran with args: {}'.format(args))
        logging.info(orig_func(*args))
        return orig_func(*args)

    return wrapper

def my_timer(orig_func):
    import time
    import logging
    logging.basicConfig(filename='Testdatenfile.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args):
        t1 = time.time()
        result = orig_func(*args)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        logging.info('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

In [14]:
@my_logger
@my_timer
def fit(x_train, y_train): 
    return model.fit(x_train, y_train)
    
@my_logger
@my_timer
def predict(x_test):
    return model.predict(x_test)

In [15]:
print(fit(x_train, y_train))

print(predict(x_test))

1563/1563 [==============================] - 6s 4ms/step - loss: 0.0603 - sparse_categorical_accuracy: 0.9821
fit ran in: 6.685071706771851 sec
1563/1563 [==============================] - 6s 4ms/step - loss: 0.0533 - sparse_categorical_accuracy: 0.9843
fit ran in: 6.5847601890563965 sec
predict ran in: 0.6255404949188232 sec
predict ran in: 0.5951237678527832 sec
[[1.27383489e-14 1.19200275e-11 2.35281359e-08 ... 9.99998450e-01
  3.81870143e-12 2.15759655e-09]
 [2.15352303e-15 1.04889009e-09 1.00000000e+00 ... 3.11272347e-15
  3.59436900e-13 5.49058709e-19]
 [1.11915730e-08 9.99228597e-01 3.06771108e-05 ... 5.35879808e-04
  1.26411454e-04 9.25061101e-07]
 ...
 [2.65568194e-13 1.19467871e-13 2.91245045e-12 ... 3.90618510e-07
  1.95280847e-07 1.36016679e-04]
 [2.58637716e-19 2.91406518e-16 1.85969723e-17 ... 4.95089021e-15
  5.11160891e-10 1.19346553e-14]
 [3.20318585e-16 2.90695648e-18 4.95527881e-13 ... 1.00524975e-18
  8.15504348e-15 9.68625072e-20]]


# Gut gemacht!